# Text segmentation using Hidden Markov Models

In [41]:
import os
import glob
import numpy as np
from visualize_segmentation import visualize_segmentation

In [48]:
ROOT = os.path.abspath('/Users/elie/Desktop/TelecomParis/S8/SD-TSIA214/Data and scripts for HMM Training Session-20240312/')

PERL_DIR = os.path.join(ROOT,'PerlScriptAndModel')
RES_DIR = os.path.join(ROOT,'Viterbi_Res')
SEGMENTED_DIR = os.path.join(ROOT, 'segmented')

### Question 1

Since it is assumed that each email contains a header and the decoding necessarily begins in state 1, the initial probability vector, denoted as $\pi$, will be:

$\pi = \begin{pmatrix}
1 \\
0  
 \end{pmatrix}$

### Question 2

The probability to move from state 1 to state 2 (transitioning from header to body) is given by the element in the first row and second column, which is approximately 0.000781921964187974.
The probability to remain in state 2 (stay in the body) is given by the element in the second row and second column, which is 1.
The higher probability is to remain in state 2 (body) because once the model enters the body state, it stays there for the entirety of the email since each email contains exactly one body. Therefore, the transition probability from state 2 to itself is 1.
The lower probability to move from state 2 to state 1 (from body to header) because an email have only one header and one body and as seen before the state 1 is the beginning state, so if we are in state 2 we can't move to state 1 because we already reach it in the intial state.

### Question3

The size of the matrix B is determined by the number of states (2) and the number of different characters $N$. Since each state (header and body) is characterized by a discrete probability distribution on the characters, B will have 2 rows (corresponding to the states) and $N$ columns (corresponding to the characters).
So, the size of B is $2\times N$. Here $N=256$ because it is ASCII.
Therefore, the size is $2\times 256$.

### Coding/Decoding Mails

In [39]:
DATA_DIR = os.path.join(ROOT,'dat')

# Iterate through files and load the text
def files_iter(data_dir, with_name=False):
    files = glob.glob('{}/*.dat'.format((data_dir)))
    if with_name:
        for f in files:
            # Get the filename
            name = f.split('/')[-1].split('.')[0]
            # Return filename and associated text
            yield name, np.loadtxt(f)
    else:
        for f in files :
            yield np.loadtxt(f)

In [40]:
# And we get a generator that will allow us to iterate through the mails
mail_iter = files_iter(DATA_DIR, with_name=True)

### Distribution files

In [5]:
PERL_DIR = '/Users/elie/Desktop/TelecomParis/S8/SD-TSIA214/Data and scripts for HMM Training Session-20240312/PerlScriptAndModel/'

# Writing a function to get the probability data
def get_emission_prob(perl_dir):
    prob_data = np.loadtxt(os.path.join(perl_dir, 'P.text'))
    return prob_data

In [6]:
# Inputs to the Viterbi function
trans = np.array([[0.999218078035812, 0.000781921964187974], [0, 1]]) #give in the subject
emission_prob = get_emission_prob(PERL_DIR) #calculate in the previous question
states = [1, 2] #give inthe subject
start_prob = [1, 0] #question 1

### To implement:

In [7]:
# Viterbi function
def viterbi(obs, states, start_prob, trans, emission_prob):
    """
        Viterbi Algorithm Implementation

        Keyword arguments:
            - obs: sequence of observation
            - states:list of states
            - start_prob:vector of the initial probabilities
            - trans: transition matrix
            - emission_prob: emission probability matrix
        Returns:
            - seq: sequence of state
    """

    # Avoid underflow: use the logarithm !
    # Avoid 0 in logarithm: use a small constant !
    small = 1e-10
    start_prob = [np.log(p + small) for p in start_prob]
    trans = np.log(trans + small)
    emission_prob = np.log(emission_prob + small)

    T = len(obs) # Number of observations
    N = len(states) # Number of model states

    # Initialisation
    log_l = np.zeros((N, T))
    bcktr = np.zeros((N, T))
    # Viterbi
    # Forward loop:
    log_l[:, 0]= start_prob + emission_prob[int(obs[0]), :]
    for t in range(1, T):
        for j in range(N):
            log_l[j, t] = np.max(log_l[:, t - 1] + trans[:, j]) + emission_prob[int(obs[t]), j]
            bcktr[j, t] = np.argmax(log_l[:, t - 1] + trans[:, j] + emission_prob[int(obs[t]), j])
    # Backward loop
    path = np.zeros(T, dtype=int)
    path[-1] = np.argmax(log_l[:, -1])
    for i in range(T - 2, -1, -1):
        path[i] = bcktr[path[i + 1], i + 1]

    return path

In [14]:
RES_DIR = '/Users/elie/Desktop/TelecomParis/S8/SD-TSIA214/Data and scripts for HMM Training Session-20240312/Viterbi_Res'

# Creating a directory to put the result of the viterbi function
if not os.path.exists(RES_DIR):
    os.mkdir(RES_DIR)

# Function that will write a viterbi path for a mail in a dedicated result file
def create_viterbi_path_file(mail_name, viterbi_path):
    with open('{}/{}_path.txt'.format(RES_DIR, mail_name), 'w') as f:
        f.write(''.join([str(c) for c in viterbi_path]))

In [15]:
# Using our generator, we get the mail names and data
for name_file, data in mail_iter:
    # Find out the viterbi path using viterbi
    viterbi_path = viterbi(data, states, start_prob, trans, emission_prob)
    file_path  = '/Users/elie/Desktop/TelecomParis/S8/SD-TSIA214/Data and scripts for HMM Training Session-20240312/Viterbi_Res' + str(name_file[:-4]) + '_path.txt'
    if not os.path.exists(file_path):
        print("File : ", name_file, "isnt in the folder")
    # Put it in the result file
    create_viterbi_path_file(name_file, viterbi_path)

Le mail 11 et le mail 30 font partie des seuls où on fait une transition 1 vers 0, ce qui a d'après notre code uen ifnime probabilité de se faire.

### Visualizing segmentation

In [44]:
# Writing a function to go into the directory and execute the perl script "segment.pl" on the mail in the given path
def exec_perl_script(mail, path):
    res = !cd {PERL_DIR}; perl segment.pl {mail} {path}
    return res

# Writing a function getting the original mail, the result of viterbi, and applying the segmentation script
# Then putting the result
def segment_mail(mail_name, data_dir, output_dir):
    # Get the full path of the mail
    mail = os.path.join(data_dir, mail_name + '.txt')
    # Get the full path of the result
    path = os.path.join(output_dir, mail_name + '_path.txt')
    # Execute the visualization script
    formatted_mail = exec_perl_script(mail, path)
    # Get the results
    formatted_mail_text = open(path).readlines()
    # Go through the resulting text until the cutting line
    for i, line in enumerate(formatted_mail_text):
        # If this was not the last line, return the text cut in to parts: header and body
        # If not, it's just a header
        print("blabla: ",line.strip())
        if line.strip() == "===== cut here":
            # If this was not the last line, return the text cut in to parts: header and body
            print("ca cut")
            header = formatted_mail_text[:i]
            body = formatted_mail_text[i+1:]
            return header, body
    return formatted_mail_text, []

In [49]:
def segment_mail_new(mail_name, data_dir, output_dir):
    # Get the full path of the mail
    mail = os.path.join(data_dir, mail_name + '.txt')
    # Get the full path of the result
    path = os.path.join(output_dir, mail_name + '_path.txt')
    # Execute the visualization script
    # Read the result
    with open(path, 'r') as f:
        path_read = f.read()
        path = [int(c) for c in path_read]

    path = np.array(path)

    # Get the full path of the segmented mail
    segmented_filename = os.path.join(SEGMENTED_DIR, '{}_segmented.txt'.format(mail_name))
    # Execute the segmentation script
    visualize_segmentation(mail, path, segmented_filename)
    # Get the results
    with open(segmented_filename, 'r') as f:
        segmented_mail = f.read()
    # Split the segmented mail into header and body based on the cutting line
    cutting_line = "===== cut here"
    if cutting_line in segmented_mail:
        header, body = segmented_mail.split(cutting_line, 1)
        return header, body
    else:
        return segmented_mail, None

In [54]:
# Getting mails names
mail_iter = files_iter(DATA_DIR, with_name=True)

if not os.path.exists(SEGMENTED_DIR):
    os.mkdir(SEGMENTED_DIR)

# Call the function and look at the result of segmentation
for name_file, data in mail_iter:
    if name_file in ['mail11', 'mail30']:
        header, body = segment_mail_new(name_file, DATA_DIR, RES_DIR)
        print("Mail:", name_file)
        print("Header:", "".join(header))
        print("Body:", "".join(body))
        print("\n")

Mail: mail30
Header: From ilug-admin@linux.ie  Fri Aug 23 11:07:51 2002
Return-Path: <ilug-admin@linux.ie>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id 7419C4416C
	for <zzzz@localhost>; Fri, 23 Aug 2002 06:06:33 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Fri, 23 Aug 2002 11:06:33 +0100 (IST)
Received: from lugh.tuatha.org (root@lugh.tuatha.org [194.125.145.45]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MJtgZ22471 for
    <zzzz-ilug@spamassassin.taint.org>; Thu, 22 Aug 2002 20:55:42 +0100
Received: from lugh (root@localhost [127.0.0.1]) by lugh.tuatha.org
    (8.9.3/8.9.3) with ESMTP id UAA19436; Thu, 22 Aug 2002 20:53:00 +0100
X-Authentication-Warning: lugh.tuatha.org: Host root@localhost [127.0.0.1]
    claimed to be lugh
Received: from mail02.svc.cra.dublin.eircom.net
    (mail02.svc.cra

### Question 5

The only modification  will be the size of the matrix. In the case of Separating in 3 parts (header, body, signature), we will have:

$$\pi = 
\begin{pmatrix} 
1\\
0\\
0
\end{pmatrix}$$

owing to an email can't begin directmly by the body or the signature.
We can also give a possible expression for the transsition matrix:

$$A = 
\begin{pmatrix}
a_{11} & a_{12} & 0\\
0 & a_{22} & a_{23}\\
0 & 0 & 1\\
\end{pmatrix}
$$

Indeed, $a_{13}$ is equal to zero because it is impossible to move from header to signature without body as an intermediary state.
In the same way as before, it is impossible to go back from body to header.
And when you are in the signature, you can't go back to header or even body.

Another change, will be the matrix B which his size become $256\times3$.

### Question 6

We need a 4th state, mail_included, which comes before the signature of the mail. There are few modifications on initial probability vector and transition matrix:

$$\pi = 
\begin{pmatrix}
1\\
0\\
0\\
0\\
\end{pmatrix}$$

$$ A = \begin{bmatrix} 
p_{11} & p_{12} & p_{13} & 0 \\
0 & p_{22} & p_{23} & p_{24}\\
0 & p_{32} & p_{33} & p_{34}\\
0 & 0 & 0 & 1
\end{bmatrix}$$

Moreover, knowing that the mail included always starts with character ">", the probability of the character being on state 3 is higher than in the other states. This information should be included in the conditional probability matrix `B` which size become $256\times4$.